# LangGraph 201: Building an Email Agent

In this notebook, we're going to walk through setting up an **email agent** in LangGraph. We will start from a simple ReAct-style agent and add additional steps into the workflow, simulating a realistic email assistant.

![Arch](../../images/email_agent.png) 

For a deeper dive into LangGraph primitives and learning our framework, check out our [LangChain Academy](https://academy.langchain.com/courses/intro-to-langgraph)!


## Pre-work: Setup

### Loading Environment Variables

To start, let's load our environment variables from our .env file. Make sure all of the keys necessary in .env.example are included!

> **⚠️ Important Notes:**
> - **Before running this notebook**, make sure you've run `pnpm install` from the project root
> - **Wait a few seconds** between running cells to avoid tslab timing issues
> - If you get a "rebuildTimer" error, **restart the kernel** and try again


In [ ]:
// Load environment variables
try {
    await import("dotenv/config");
    console.log("✓ Environment loaded successfully!");
} catch (error) {
    console.log("⚠️  Could not load dotenv. Make sure you've run 'pnpm install' from the project root.");
}

console.log("\n📝 Make sure OPENAI_API_KEY is set in your .env file or environment");


### Setting up Short and Long-Term Memory

We will also initialize a checkpointer for **short-term memory**, maintaining context within a single thread. 

**Long term memory** lets you store and recall information between conversations. Today, we will utilize our long term memory store to store user preferences for personalization.


In [ ]:
import { MemorySaver, InMemoryStore } from "@langchain/langgraph";

// Initializing long term memory store 
const inMemoryStore = new InMemoryStore();

// Initializing checkpoint for thread-level memory 
const checkpointer = new MemorySaver();

console.log("✓ Memory stores initialized!");


### Initialize our LLM


In [ ]:
import { initChatModel } from "langchain";

const llm = await initChatModel("openai:gpt-4o-mini", { temperature: 0.0 });

console.log("✓ Model initialized!");


## Part 1: Creating an Email Agent
![Arch](../../images/email_response_agent.png)


### State

How does information flow through the steps?  

State is the first LangGraph concept we'll cover. **State can be thought of as the memory of the agent - its a shared data structure that's passed on between the nodes of your graph**, representing the current snapshot of your application. 

For this our email agent our state will track the following elements: 
1. An input email
2. A classification decision - whether to respond to the email
3. The conversation history
4. Any loaded long-term memories
5. Remaining steps - before we hit our recursion limit


In [ ]:
import { Annotation, messagesStateReducer } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const StateAnnotation = Annotation.Root({
    email_input: Annotation<Record<string, any>>,
    classification_decision: Annotation<"ignore" | "respond" | "notify">,
    messages: Annotation<BaseMessage[]>({
        reducer: messagesStateReducer,
        default: () => [],
    }),
    loaded_memory: Annotation<string>,
    remaining_steps: Annotation<number>,
});

type State = typeof StateAnnotation.State;

console.log("✓ State defined!");


### Helper Functions

For email processing, let's define helpers to parse and format email inputs.


In [ ]:
function parseEmail(emailInput: Record<string, any>): [string, string, string, string] {
    return [
        emailInput.author,
        emailInput.to,
        emailInput.subject,
        emailInput.email_thread,
    ];
}

function formatEmailMarkdown(
    subject: string,
    author: string,
    to: string,
    emailThread: string,
    emailId?: string
): string {
    const idSection = emailId ? `\n**ID**: ${emailId}` : "";
    
    return `

**Subject**: ${subject}
**From**: ${author}
**To**: ${to}${idSection}

${emailThread}

---
`;
}

console.log("✓ Helper functions defined!");


### Tools

Let's define a list of **tools** our agent will have access to. Tools are functions that can act as extension of the LLM's capabilities. In our case, we will first create several tools for managing emails and calendar.

We can create tools using the `tool` function to create a tool.


In [ ]:
import { tool } from "langchain";
import { z } from "zod";

const scheduleMeeting = tool(
    async ({ attendees, subject, durationMinutes, preferredDay, startTime }) => {
        // Placeholder response - in real app would check calendar and schedule
        const date = new Date(preferredDay);
        const dateStr = date.toLocaleDateString('en-US', { 
            weekday: 'long', 
            year: 'numeric', 
            month: 'long', 
            day: 'numeric' 
        });
        return `Meeting '${subject}' scheduled on ${dateStr} at ${startTime} for ${durationMinutes} minutes with ${attendees.length} attendees`;
    },
    {
        name: "schedule_meeting",
        description: "Schedule a calendar meeting.",
        schema: z.object({
            attendees: z.array(z.string()).describe("List of attendee email addresses"),
            subject: z.string().describe("Meeting subject"),
            durationMinutes: z.number().describe("Duration in minutes"),
            preferredDay: z.string().describe("Preferred day for the meeting (ISO date string)"),
            startTime: z.number().describe("Start time (hour in 24-hour format)"),
        }),
    }
);

const checkCalendarAvailability = tool(
    async ({ day }) => {
        // Placeholder response - in real app would check actual calendar
        return `Available times on ${day}: 9:00 AM, 2:00 PM, 4:00 PM`;
    },
    {
        name: "check_calendar_availability",
        description: "Check calendar availability for a given day.",
        schema: z.object({
            day: z.string().describe("Day to check availability (ISO date string)"),
        }),
    }
);

const writeEmail = tool(
    async ({ to, subject, content }) => {
        // Placeholder response - in real app would send email
        return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
    },
    {
        name: "write_email",
        description: "Write and send an email.",
        schema: z.object({
            to: z.string().describe("Recipient email address"),
            subject: z.string().describe("Email subject"),
            content: z.string().describe("Email content"),
        }),
    }
);

const Done = tool(
    async ({ done }) => {
        return "E-mail has been sent.";
    },
    {
        name: "Done",
        description: "E-mail has been sent.",
        schema: z.object({
            done: z.boolean().describe("Whether the task is complete"),
        }),
    }
);

const tools = [scheduleMeeting, checkCalendarAvailability, writeEmail, Done];

console.log("✓ Tools defined!");
console.log(`  - ${tools.map(t => t.name).join(", ")}`);


To make our LLM aware that these tools are available to call, we'll use the `bindTools` method.


In [ ]:
const toolsByName = Object.fromEntries(tools.map(t => [t.name, t]));

const llmWithTools = llm.bindTools(tools, { 
    tool_choice: "any",
});

console.log("✓ LLM bound with tools!");


### Nodes

Now that we have a list of tools, we are ready to build nodes that interact with them. 

Nodes are just TypeScript functions. Nodes take in your graph's State as input, execute some logic, and return a new State. 

Here, we're just going to set up 2 nodes for our ReAct agent:
1. **reasoning**: Reasoning node that decides which function to invoke 
2. **tools**: Node that contains all the available tools and executes the function

LangGraph has a `ToolNode` that we can utilize to create a node for our tools.


In [ ]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

// Node
const toolNode = new ToolNode(tools);

console.log("✓ Tool node created!");


For our reasoning node, we'll need to create a prompt that instructs the agent on how to handle emails.


In [ ]:
import { SystemMessage, HumanMessage } from "@langchain/core/messages";

function createAgentPrompt(state: State): BaseMessage[] {
    const today = new Date().toISOString().split('T')[0];
    
    const actionInstructions = `
    < Role >
    You are a top-notch executive assistant who cares about helping your executive perform as well as possible.
    </ Role >

    < Tools >
    You have access to the following tools to help manage communications and schedule:

    1. write_email(to, subject, content) - Send emails to specified recipients
    2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings
    3. check_calendar_availability(day) - Check available time slots for a given day
    4. Done - E-mail has been sent
    </ Tools >

    < Instructions >
    When handling emails, follow these steps:
    1. Carefully analyze the email content and purpose
    3. For responding to the email, draft a response email with the write_email tool
    4. For meeting requests, use the check_calendar_availability tool to find open time slots
    5. To schedule a meeting, use the schedule_meeting tool with a datetime object for the preferred_day parameter
    - Today's date is ${today} - use this for scheduling meetings accurately
    6. If you scheduled a meeting, then draft a short response email using the write_email tool
    7. After using the write_email tool, the task is complete
    8. If you have sent the email, then use the Done tool to indicate that the task is complete
    </ Instructions >

    < Background >
    I'm Robert, a software engineer at LangChain.
    </ Background >

    < Response Preferences >
    Use professional and concise language. If the e-mail mentions a deadline, make sure to explicitly acknowledge and reference the deadline in your response.
    </ Response Preferences >
    `;
    
    const [author, to, subject, emailThread] = parseEmail(state.email_input);
    const emailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);
    const emailRequest = `Please handle the following email:\n${emailMarkdown}`;
    
    const prompt = [
        new SystemMessage(actionInstructions),
        new HumanMessage(emailRequest),
        ...state.messages
    ];
    
    return prompt;
}

console.log("✓ Prompt function created!");


Now let's create the reasoning node that will use this prompt.


In [ ]:
// Reasoning node
async function reasoningNode(state: State) {
    const prompt = createAgentPrompt(state);
    const response = await llmWithTools.invoke(prompt);
    return { messages: [response] };
}

console.log("✓ Reasoning node created!");


### Edges

We'll need conditional edges to determine when to continue tool calling and when to stop.


In [ ]:
import { END } from "@langchain/langgraph";

function shouldContinue(state: State): "tools" | typeof END {
    const messages = state.messages;
    const lastMessage = messages[messages.length - 1];
    
    // If there are tool calls, continue to tools node
    if (lastMessage.additional_kwargs?.tool_calls && lastMessage.additional_kwargs.tool_calls.length > 0) {
        return "tools";
    }
    
    // Otherwise, end
    return END;
}

console.log("✓ Conditional edge function created!");


### Graph

Let's build the graph!


In [ ]:
import { StateGraph, START } from "@langchain/langgraph";

const agentBuilder = new StateGraph(StateAnnotation);

// Add nodes
agentBuilder.addNode("agent", reasoningNode);
agentBuilder.addNode("tools", toolNode);

// Add edges to connect nodes
agentBuilder.addEdge(START, "agent");
agentBuilder.addConditionalEdges(
    "agent",
    shouldContinue,
    {
        // Name returned by shouldContinue : Name of next node to visit
        tools: "tools",
        [END]: END,
    }
);
agentBuilder.addEdge("tools", "agent");

// Compile the agent
const agent = agentBuilder.compile({ checkpointer, store: inMemoryStore });

console.log("✓ Agent compiled!");


### Testing

Let's see how our agent responds to emails!


In [ ]:
import { v4 as uuidv4 } from "uuid";

const threadId = uuidv4();
const config = { configurable: { thread_id: threadId } };

const emailInput = {
    to: "Robert Xu <Robert@company.com>",
    author: "Team Lead <teamlead@company.com>",
    subject: "Quarterly planning meeting",
    email_thread: "Hi Robert,\n\nIt's time for our quarterly planning session. I'd like to schedule a 90-minute meeting next week to discuss our roadmap for Q3.\n\nCould you let me know your availability for Monday or Wednesday? Ideally sometime between 10AM and 3PM.\n\nLooking forward to your input on the new feature priorities.\n\nBest,\nTeam Lead"
};

const result = await agent.invoke({ email_input: emailInput }, config);

console.log("Responding to email: ");
console.log(formatEmailMarkdown(emailInput.subject, emailInput.author, emailInput.to, emailInput.email_thread));

for (const message of result.messages) {
    console.log("\n" + "=".repeat(50));
    console.log(`${message._getType()} Message`);
    if (message.additional_kwargs?.tool_calls) {
        console.log("Tool Calls:");
        for (const toolCall of message.additional_kwargs.tool_calls) {
            console.log(`  ${toolCall.function.name} (${toolCall.id})`);
            console.log(`  Args: ${toolCall.function.arguments}`);
        }
    } else if (message.content) {
        console.log(message.content);
    }
}


## Part 2: Using a Prebuilt

The architecture we created for our email agent is a common architecture known as a ReAct agent. This tool-calling format is very popular, so LangChain actually provides a prebuilt function to easily spin up ReAct agents.

Let's reimplement our agent using the prebuilt!


In [ ]:
import { createAgent } from "langchain";

// Define the subagent 
const emailPrebuilt = createAgent({
    model: llm,
    tools: tools,
    name: "email_prebuilt",
    systemPrompt: createAgentPrompt,
    stateSchema: StateAnnotation,
    checkpointer: checkpointer,
    store: inMemoryStore
});

console.log("✓ Prebuilt agent created!");


Let's go ahead and test our prebuilt!


In [ ]:
const threadId2 = uuidv4();
const config2 = { configurable: { thread_id: threadId2 } };

const emailInput2 = {
    to: "Robert Xu <Robert@company.com>",
    author: "Team Lead <teamlead@company.com>",
    subject: "Quarterly planning meeting",
    email_thread: "Hi Robert,\n\nIt's time for our quarterly planning session. I'd like to schedule a 90-minute meeting next week to discuss our roadmap for Q3.\n\nCould you let me know your availability for Monday or Wednesday? Ideally sometime between 10AM and 3PM.\n\nLooking forward to your input on the new feature priorities.\n\nBest,\nTeam Lead"
};

const result2 = await emailPrebuilt.invoke({ email_input: emailInput2 }, config2);

console.log("Responding to email: ");
console.log(formatEmailMarkdown(emailInput2.subject, emailInput2.author, emailInput2.to, emailInput2.email_thread));

for (const message of result2.messages.slice(-5)) {  // Show last 5 messages
    console.log("\n" + "=".repeat(50));
    console.log(`${message._getType()} Message`);
    if (message.additional_kwargs?.tool_calls) {
        console.log("Tool Calls:");
        for (const toolCall of message.additional_kwargs.tool_calls) {
            console.log(`  ${toolCall.function.name}`);
        }
    } else if (message.content) {
        const contentStr = typeof message.content === 'string' ? message.content : JSON.stringify(message.content);
        console.log(contentStr.substring(0, 200));
    }
}


## Part 3: Email Triage + Human-in-the-Loop

Now that we have a working email agent, let's add a triage step that classifies emails before responding. We'll also add a human-in-the-loop step for emails that need verification.

![Arch](../../images/email_triage.png)


### Router Schema

First, let's define a schema for our email classification router using structured output.


In [ ]:
const RouterSchema = z.object({
    reasoning: z.string().describe("Step-by-step reasoning behind the classification."),
    classification: z.enum(["ignore", "respond", "notify"]).describe(
        "The classification of an email: 'ignore' for irrelevant emails, " +
        "'notify' for important information that doesn't need a response, " +
        "'respond' for emails that need a reply"
    ),
});

const llmRouter = llm.withStructuredOutput(RouterSchema);

console.log("✓ Router schema and LLM created!");


We'll then create the triage node itself, alongside its prompt.


In [ ]:
function createTriagePrompt(state: State): BaseMessage[] {
    const loadedMemory = state.loaded_memory || "";

    const triageInstructions = `
    < Role >
    Your role is to triage incoming emails based upon instructs and background information below.
    </ Role >

    < Background >
    I'm Robert, a software engineer at LangChain.
    </ Background >

    < Instructions >
    Categorize each email into one of three categories:
    1. IGNORE - Emails that are not worth responding to or tracking
    2. NOTIFY - Important information that worth notification but doesn't require a response
    3. RESPOND - Emails that need a direct response
    Classify the below email into one of these categories.
    </ Instructions >

    < Rules >
    Emails that are not worth responding to:
    - Marketing newsletters and promotional emails
    - Spam or suspicious emails
    - CC'd on FYI threads with no direct questions

    There are also other things that should be known about, but don't require an email response. For these, you should notify (using the \`notify\` response). Examples of this include:
    - Team member out sick or on vacation
    - Build system notifications or deployments
    - Project status updates without action items
    - Important company announcements
    - FYI emails that contain relevant information for current projects
    - HR Department deadline reminders
    - GitHub notifications

    Emails that are worth responding to:
    - Direct questions from team members requiring expertise
    - Meeting requests requiring confirmation
    - Critical bug reports related to team's projects
    - Requests from management requiring acknowledgment
    - Client inquiries about project status or features
    - Technical questions about documentation, code, or APIs (especially questions about missing endpoints or features)
    - Personal reminders related to family (wife / daughter)
    - Personal reminder related to self-care (doctor appointments, etc)
    </ Rules >

    ${loadedMemory}
    `;
    
    const [author, to, subject, emailThread] = parseEmail(state.email_input);
    const emailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);
    const emailRequest = `Please determine how to handle the below email thread: ${emailMarkdown}`;
    
    const prompt = [
        new SystemMessage(triageInstructions),
        new HumanMessage(emailRequest),
        ...state.messages
    ];
    
    return prompt;
}

async function triageRouter(state: State) {
    /**
     * Analyze email content to decide if we should respond, notify, or ignore.
     */
    const prompt = createTriagePrompt(state);
    // Run the router LLM
    const result = await llmRouter.invoke(prompt);

    // Decision
    const classification = result.classification;
    return { classification_decision: classification };
}

console.log("✓ Triage router created!");


In [ ]:
import { interrupt } from "@langchain/langgraph";

// Node
function humanInput(state: State) {
    /** Node to incorporate human feedback */
    const [author, to, subject, emailThread] = parseEmail(state.email_input);
    const emailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);
    const userInput = interrupt(`Please determine whether the following email deserves a response (Y/n): ${emailMarkdown}`);

    const log = "Email originally marked as notify, but user flagged it as a scenario where a response is warranted.";
    if (String(userInput).toLowerCase() === "y") {
        return { 
            classification_decision: "respond" as const, 
            messages: [new HumanMessage(log)]
        };
    } else {
        return { classification_decision: "ignore" as const };
    }
}

console.log("✓ Human input node created!");


Finally, we'll define the new edges that we'll need. We'll define an edge that triggers Human In the Loop if the triage step returns a `notify` classification, as well as an edge from our human feedback step to the rest of the agent.


In [ ]:
function handleClassification(state: State): "human_input" | "email_agent" | typeof END {
    /** Trigger human review if the email is classified as notify */
    if (state.classification_decision === "notify") {
        return "human_input";
    } else if (state.classification_decision === "respond") {
        return "email_agent";
    } else {
        return END;
    }
}

function handleHumanInput(state: State): "email_agent" | typeof END {
    /** Handle human input */
    if (state.classification_decision === "respond") {
        return "email_agent";
    } else {
        return END;
    }
}

console.log("✓ Conditional edge functions created!");


Let's compile our agent!


In [ ]:
const emailHitlWorkflow = new StateGraph(StateAnnotation);
emailHitlWorkflow.addNode("triage", triageRouter);
emailHitlWorkflow.addNode("human_input", humanInput);
emailHitlWorkflow.addNode("email_agent", agent);
emailHitlWorkflow.addEdge(START, "triage");
emailHitlWorkflow.addConditionalEdges(
    "triage",
    handleClassification,
    {
        human_input: "human_input",
        email_agent: "email_agent",
        [END]: END,
    }
);
emailHitlWorkflow.addConditionalEdges(
    "human_input",
    handleHumanInput,
    {
        email_agent: "email_agent",
        [END]: END,
    }
);

const emailHitl = emailHitlWorkflow.compile({ checkpointer, store: inMemoryStore });

console.log("✓ Email agent with HITL compiled!");


### Testing

Let's invoke our new agent!


In [ ]:
const threadId3 = uuidv4();
const config3 = { configurable: { thread_id: threadId3 } };

const emailInput3 = {
    to: "Robert Xu <Robert@company.com>",
    author: "SysAdmin <sysadmin@company.com>",
    subject: "Scheduled maintenance - database downtime",
    email_thread: "Hi team,\n\nJust a reminder that we have scheduled maintenance tonight from 2AM to 4AM EST. The main database will be unavailable during this window.\n\nPlease plan accordingly and avoid any critical deployments during this time.\n\nThanks,\nSysAdmin Team"
};

const result3 = await emailHitl.invoke({ email_input: emailInput3 }, config3);

console.log("\n" + "=".repeat(50));
console.log("Classification:", result3.classification_decision);

if ("__interrupt__" in result3) {
    console.log("\n🛑 Agent paused for human input");
    console.log("Interrupt details:", result3.__interrupt__);
}


Now let's resume with human approval:


In [ ]:
import { Command } from "@langchain/langgraph";

// Resume with approval
const result4 = await emailHitl.invoke(
    new Command({ resume: "y" }),
    config3
);

console.log("\n" + "=".repeat(50));
console.log("After human approval:");
console.log("Classification:", result4.classification_decision);
console.log("Number of messages:", result4.messages.length);

// Show last few messages
for (const message of result4.messages.slice(-3)) {
    console.log("\n" + "-".repeat(30));
    console.log(`${message._getType()} Message`);
    if (message.content) {
        const contentStr = typeof message.content === 'string' ? message.content : JSON.stringify(message.content);
        console.log(contentStr.substring(0, 150));
    }
}


## [Optional] Part 4: Memory

Now that we have created an agent workflow that includes verification and execution, let's take it a step further. 

**Long term memory** lets you store and recall information between conversations. We have already initialized a long term memory store. 

![memory](../../images/email_agent_memory.png)

In this step, we will add 2 nodes: 
- **load_memory** node that loads from the long term memory store
- **create_memory** node that saves any preferences that the customer has shared about themselves


Let's start with load memory. This will allow us to personalize what emails we want our agent to respond to, without hard-coding it into the prompt.


In [ ]:
import type { RunnableConfig } from "@langchain/core/runnables";

// Helper function to structure memory 
function formatUserMemory(userData: any): string {
    /**Formats preferences from users, if available.*/
    const profile = userData.memory;
    let result = "<Additional Rules>\n";
    result += "The following are custom rules the user has noted to be important. Please prioritize these rules:";
    if (profile?.response_preferences && profile.response_preferences.length > 0) {
        result += "\n- " + profile.response_preferences.join("\n- ");
    }
    result += "\n</Additional Rules>";
    
    return result.trim();
}

// Node
async function loadMemory(state: State, config: RunnableConfig) {
    /**Loads preferences from users, if available.*/
    
    const namespace = ["memory_profile", "Robert"];
    const store = config.store;
    if (!store) {
        return { loaded_memory: "" };
    }
    
    const existingMemory = await store.get(namespace, "user_memory");
    let formattedMemory = "";
    if (existingMemory && existingMemory.value) {
        formattedMemory = formatUserMemory(existingMemory.value);
    }

    return { loaded_memory: formattedMemory };
}

console.log("✓ Load memory node created!");


Next, we'll make our create_memory node. We'll use structured output to ensure all our memories are in the same format.


In [ ]:
// User profile structure for creating memory
const UserProfile = z.object({
    response_preferences: z.array(z.string()).describe(
        "A list of rules describing what types of email the user would like to respond to"
    ),
});

console.log("✓ User profile schema created!");


In [ ]:
const createMemoryPrompt = `You are an expert analyst that is observing a conversation that has taken place between a customer and an executive assistant. The executive assistant helps the customer handle their emails.
You are tasked with analyzing the interaction that has taken place between the customer and the executive assistant, and updating the memory profile associated with the customer. 
You specifically care about saving any preferences the customer has shared about themselves, particularly their email response preferences to their memory profile.

<core_instructions>
1. The memory profile may be empty. If it's empty, you should ALWAYS create a new memory profile for the customer.
2. You should identify what characteristics about the email resulted in the user wanting to respond to it.
3. For each key in the memory profile, if there is no new information, do NOT update the value - keep the existing value unchanged.
4. ONLY update the values in the memory profile if there is new information.
</core_instructions>

<expected_format>
The customer's memory profile should have the following fields:
- response_preferences: a list of rules describing what types of email the user would like to respond to

IMPORTANT: ENSURE your response is an object with these fields.
</expected_format>

<important_context>
**IMPORTANT CONTEXT BELOW**
To help you with this task, I have attached the conversation that has taken place between the customer and the customer support assistant below, as well as the existing memory profile associated with the customer that you should either update or create. 

The conversation between the customer and the customer support assistant that you should analyze is as follows:
{conversation}

The existing memory profile associated with the customer that you should either update or create based on the conversation is as follows:
{memory_profile}

</important_context>

Reminder: Take a deep breath and think carefully before responding.
`;

// Node
async function createMemory(state: State, config: RunnableConfig) {
    const namespace = ["memory_profile", "Robert"];
    const formattedMemory = state.loaded_memory || "";

    const emailInput = state.email_input;
    const [author, to, subject, emailThread] = parseEmail(emailInput);
    const formattedEmail = formatEmailMarkdown(subject, author, to, emailThread);
    const initialMessage = `Initial email received: ${formattedEmail}\n`;
    const conversation = [new HumanMessage(initialMessage), ...state.messages];

    const formattedSystemMessage = new SystemMessage(
        createMemoryPrompt
            .replace("{conversation}", JSON.stringify(conversation.map(m => ({ type: m._getType(), content: m.content }))))
            .replace("{memory_profile}", formattedMemory)
    );
    
    const updatedMemory = await llm.withStructuredOutput(UserProfile).invoke([formattedSystemMessage]);
    const key = "user_memory";
    
    const store = config.store;
    if (store) {
        await store.put(namespace, key, { memory: updatedMemory });
        console.log("✓ Memory saved:", updatedMemory);
    }
    
    return {};
}

console.log("✓ Create memory node created!");


Let's add these nodes to our graph. We'll only create a new memory when the user has offered feedback. In other words, we'll only create a new memory when the user has decided to respond to an email originally marked as "notify only".


In [ ]:
function shouldCreateMemory(state: State): "create_memory" | typeof END {
    /** Only create a new memory if the user has decided to respond to an email */
    const messages = state.messages;

    const correction = "Email originally marked as notify, but user flagged it as a scenario where a response is warranted.";
    for (const message of messages) {
        if (message._getType() === "human" && message.content && message.content.includes(correction)) {
            return "create_memory";
        }
    }

    return END;
}

console.log("✓ Memory router created!");


Now let's build the complete workflow with memory!


In [ ]:
const emailMemoryWorkflow = new StateGraph(StateAnnotation);
emailMemoryWorkflow.addNode("triage", triageRouter);
emailMemoryWorkflow.addNode("human_input", humanInput);
emailMemoryWorkflow.addNode("email_agent", agent);

emailMemoryWorkflow.addNode("load_memory", loadMemory);
emailMemoryWorkflow.addNode("create_memory", createMemory);

emailMemoryWorkflow.addEdge(START, "load_memory");
emailMemoryWorkflow.addEdge("load_memory", "triage");

emailMemoryWorkflow.addConditionalEdges(
    "triage",
    handleClassification,
    {
        human_input: "human_input",
        email_agent: "email_agent",
        [END]: END,
    }
);
emailMemoryWorkflow.addConditionalEdges(
    "human_input",
    handleHumanInput,
    {
        email_agent: "email_agent",
        [END]: END,
    }
);
emailMemoryWorkflow.addConditionalEdges(
    "email_agent",
    shouldCreateMemory,
    {
        create_memory: "create_memory",
        [END]: END,
    }
);

emailMemoryWorkflow.addEdge("create_memory", END);

const emailAgentMemory = emailMemoryWorkflow.compile({ checkpointer, store: inMemoryStore });

console.log("✓ Email agent with memory compiled!");


### Testing the Complete Workflow

Let's test the complete workflow with memory!


In [ ]:
const threadId5 = uuidv4();
const config5 = { configurable: { thread_id: threadId5 } };

const emailInput5 = {
    to: "Robert Xu <Robert@company.com>",
    author: "HR Department <hr@company.com>",
    subject: "Benefits enrollment deadline",
    email_thread: "Hi team,\n\nFriendly reminder that the open enrollment period for health insurance and benefits ends this Friday, October 30th.\n\nPlease log into the HR portal to review and update your selections before the deadline.\n\nIf you have questions, contact us at benefits@company.com.\n\nBest,\nHR Team"
};

console.log("Testing with HR benefits email...\n");

const result5 = await emailAgentMemory.invoke({ email_input: emailInput5 }, config5);

console.log("=".repeat(50));
console.log("Classification:", result5.classification_decision);

if ("__interrupt__" in result5) {
    console.log("\n🛑 Agent paused for human input");
    
    // Resume with approval
    console.log("\nResuming with approval...\n");
    const result6 = await emailAgentMemory.invoke(
        new Command({ resume: "y" }),
        config5
    );
    
    console.log("=".repeat(50));
    console.log("Final classification:", result6.classification_decision);
    console.log("Messages processed:", result6.messages.length);
    
    // Check if memory was created
    const memoryCheck = await inMemoryStore.get(["memory_profile", "Robert"], "user_memory");
    if (memoryCheck) {
        console.log("\n✓ Memory stored successfully!");
        console.log("Preferences:", JSON.stringify(memoryCheck.value, null, 2));
    }
}


Now let's test with another email to see if the memory affects the triage decision!


In [ ]:
const threadId6 = uuidv4();
const config6 = { configurable: { thread_id: threadId6 } };

const emailInput6 = {
    to: "Robert Xu <Robert@company.com>",
    author: "HR Department <hr@company.com>",
    subject: "Annual performance review reminder",
    email_thread: "Hi Robert,\n\nThis is a reminder that your annual performance review is scheduled for next week.\n\nPlease complete your self-assessment in the HR portal by Thursday.\n\nLet me know if you have any questions.\n\nBest,\nHR Team"
};

console.log("\nTesting with another HR email (should use memory)...\n");

const result7 = await emailAgentMemory.invoke({ email_input: emailInput6 }, config6);

console.log("=".repeat(50));
console.log("Classification:", result7.classification_decision);
console.log("\nMemory should have influenced this decision!");

if ("__interrupt__" in result7) {
    console.log("🛑 Agent still paused - memory may not have full effect yet");
} else {
    console.log("✓ Agent processed without interrupt - memory working!");
}


## Summary

Congratulations! You've built a complete email agent system with:

1. **Custom StateGraph Agent** - Built from scratch with reasoning and tool nodes
2. **Prebuilt Agent** - Used LangChain's `createAgent()` for quick setup
3. **Email Triage** - Automatically classifies emails as ignore/notify/respond
4. **Human-in-the-Loop** - Pauses for human approval on notify emails
5. **Long-term Memory** - Stores and recalls user preferences across conversations

### Key Concepts Learned

- **State Management** - Using `Annotation.Root` for typed state
- **Tool Calling** - Creating tools with `tool()` and zod schemas
- **Conditional Routing** - Using conditional edges for dynamic workflows
- **Interrupts** - Pausing execution with `interrupt()` and resuming with `Command`
- **Memory Stores** - Using `InMemoryStore` for long-term memory
- **Checkpointers** - Using `MemorySaver` for conversation state

### Next Steps

- Try different email scenarios
- Customize the triage rules
- Add more tools (calendar, tasks, etc.)
- Experiment with different memory patterns
- Deploy your agent to production!
